In [1]:
#change working directory
import os
print(os.getcwd())
os.chdir('/home/student/A2/drive-download-20240507T073745Z-001')


from datetime import datetime, timedelta
import pymongo
from pymongo import MongoClient
#Residence wifi ip address: 172.16.62.212
ipadd = "10.192.33.112"

client = MongoClient(ipadd, 27017)
db = client.fit3182_db
collection = db.fit3182_assignment_db

#res = collection.find().sort('surface_temperature_celcius', pymongo.DESCENDING).limit(10)
res = collection.find().sort('date', pymongo.DESCENDING).limit(1)

for d in res:
    res = d
#     print(d)

latest_date = res['date']
#string to datetime object
# latest_date = datetime.strptime(latest_date, '%Y-%m-%d %H:%M:%S')
#string to datetime object
latest_date = datetime.strptime(latest_date, '%d/%m/%Y')
start_date = latest_date + timedelta(days=1)
print(start_date)

/home/student/A2
2024-01-01 00:00:00


In [2]:
from time import sleep
from json import dumps
from kafka3 import KafkaProducer
import random
import json
import pandas as pd

hostip = ipadd # change it to your IP

def connect_kafka_producer():
    producer = None
    
    try:
        producer = KafkaProducer(bootstrap_servers=[f'{hostip}:9092'],api_version=(0,10))
    
    except Exception as ex:
        print("Exception while connecting Kafka")
        print(str(ex))
    finally:
        return producer 
    
def publish_msg(producer, topic, data):
    try:
        #print(data)        #send the  value by dumps to JSON object
                            #then encode with utf-8 into bytes
        value_bytes = json.dumps(data).encode('utf-8')
        producer.send(topic, value=value_bytes)
        #print(data)
        """
        The flush() method in Kafka’s producer API is used to ensure that all 
        previously sent messages have been transmitted to the server and acknowledged 
        before proceeding. When you call flush(), it blocks the current thread and waits 
        for the producer to complete the sending of all records
        """
        producer.flush()  
    except Exception as ex:
        print('Exception from publish_msg func')
        print(str(ex))
    


In [3]:
"""
Event Producer 3
"""
hotspot_TERRA= pd.read_csv("./hotspot_TERRA_streaming.csv", sep=',')

hotspot_TERRA.head()

,latitude,longitude,confidence,surface_temperature_celcius
0,-37.966,145.051,78,68
1,-35.541,143.311,82,63
2,-35.554,143.307,67,53
3,-35.543,143.316,86,67
4,-37.708,145.100,80,54


In [ ]:


"""
Main Function
"""
if __name__ == '__main__':
    topic = 'Producer2'
    print('Publishing records..')
    producer03 = connect_kafka_producer()
    
    
    rand_row_indices = list(range(len(hotspot_TERRA.index)))
    random.shuffle(rand_row_indices) 
    # random choose time from 0 to 10
    max_time = 3 #testing purpose
    start_time = start_date
    
    #rand_time = list(range(1,max_time))
#     random.shuffle(rand_time) 
     

    for index, hotspot_TERRA_row in hotspot_TERRA.iterrows():
        random_time = rand_row_indices[index]%max_time + 1
        hotspot_TERRA_row = hotspot_TERRA.loc[rand_row_indices[index]]

        hotspot_TERRA_doc = {
            'latitude': hotspot_TERRA_row['latitude'],
            'longitude': hotspot_TERRA_row['longitude'],
            'confidence': hotspot_TERRA_row['confidence'],
            'surface_temperature_celcius': hotspot_TERRA_row['surface_temperature_celcius'],
            'producer': "Producer_3", #datetime ibject
            'created_time': start_time.strftime('%H:%M:%S')
        }
        #update start date
        #print(climate_doc['date'])
        #producer publish msg in strictured streaming
        # convert the dictionary to JSON-formatted string and encode as UFT-8 by publish_msg
        # msg = json.dumps(climate_doc)
        #print(climate_doc)
        #print(hotspot_TERRA_doc['random_time'])
        print(hotspot_TERRA_doc['created_time'])
        publish_msg(producer03, topic, hotspot_TERRA_doc)
        #random minute for created_time
        start_time =  start_time + timedelta(minutes=random_time)
        # feed data every random seconds
        sleep(random_time) 



Publishing records..
00:00:00
00:03:00
00:06:00
00:07:00
00:09:00
00:12:00
00:15:00
00:18:00
00:20:00
00:23:00
00:25:00
00:28:00
00:29:00
00:32:00
00:35:00
00:38:00
00:41:00
00:44:00
00:47:00
00:48:00
00:50:00
00:52:00
00:55:00
00:56:00
00:57:00
00:58:00
00:59:00
01:00:00
01:01:00
01:04:00
01:06:00
01:08:00
01:09:00
01:12:00
01:14:00
01:15:00
01:16:00
01:17:00
01:19:00
01:20:00
01:22:00
01:23:00
01:24:00
01:25:00
01:28:00
01:30:00
01:32:00
01:34:00
01:37:00
01:40:00
01:43:00
01:45:00
01:48:00
01:49:00
01:52:00
01:53:00
01:55:00
01:57:00
02:00:00
02:03:00
02:04:00
02:05:00
02:07:00
02:08:00
02:09:00
02:11:00
02:13:00
02:16:00
02:19:00
02:20:00
02:23:00
02:24:00
02:26:00
02:29:00
02:30:00
02:32:00
02:35:00
02:38:00
02:39:00
02:41:00
02:44:00
02:47:00
02:48:00
02:50:00
02:51:00
02:52:00
02:53:00
02:54:00
02:57:00
02:58:00
02:59:00
03:01:00
03:03:00
03:04:00
03:07:00
03:09:00
03:10:00
03:13:00
03:16:00
03:19:00
03:20:00
03:22:00
03:23:00
03:24:00
03:27:00
03:29:00
03:30:00
03:33:00
03:34:0

06:00:00
06:03:00
06:04:00
06:06:00
06:08:00
06:09:00
06:11:00
06:12:00
06:13:00
06:16:00
06:17:00
06:19:00
06:22:00
06:23:00
06:26:00
06:27:00
06:29:00
06:32:00
06:34:00
06:37:00
06:38:00
06:39:00
06:42:00
06:44:00
06:46:00
06:49:00
06:50:00
06:52:00
06:55:00
06:56:00
06:58:00
07:00:00
07:02:00
07:05:00
07:07:00
07:09:00
07:10:00
07:11:00
07:12:00
07:13:00
07:14:00
07:16:00
07:18:00
07:20:00
07:21:00
07:24:00
